In [1]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm_notebook
import pandas as pd
import csv
import math
from keras.utils import np_utils
%matplotlib inline

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
TrainingPercent = 80
ValidationPercent = 10
TestPercent = 10

In [3]:
def GetGSCFeatureData():
    gscSamePair = pd.read_csv('GSC-Dataset/GSC-Features-Data/same_pairs.csv') 
    gscDiffPair = pd.read_csv('GSC-Dataset/GSC-Features-Data/diffn_pairs.csv')
    #Reading human observed feature data csv 
    gscFeatureData = pd.read_csv('GSC-Dataset/GSC-Features-Data/GSC-Features.csv')
    #print(humanFeatureData.img_id)
    #print(humanSamePair.img_id_A, humanSamePair.img_id_B) 

    sameFeatureMerge=pd.merge(gscSamePair, gscFeatureData,  how='inner', left_on=['img_id_A'], right_on = ['img_id'], suffixes= ['_A','_A'])
    featureListSame=pd.merge(sameFeatureMerge, gscFeatureData, how='inner', left_on=['img_id_B'], right_on = ['img_id'], suffixes= ['_A','_B'])
    #print(featureListSame)

    diffFeatureMerge=pd.merge(gscDiffPair, gscFeatureData,  how='inner', left_on=['img_id_A'], right_on = ['img_id'], suffixes= ['_A','_A'])
    featureListDiff=pd.merge(diffFeatureMerge, gscFeatureData, how='inner', left_on=['img_id_B'], right_on = ['img_id'], suffixes= ['_A','_B'])
    return sameFeatureMerge, featureListSame, diffFeatureMerge, featureListDiff
    #print(featureListDiff.iloc[2])

In [4]:
sameFeatureMerge, featureListSame, diffFeatureMerge, featureListDiff = GetGSCFeatureData()
#Getting the array vector of Target values from 'same_pairs.csv'.
def myProcessData(featureListSame,featureListDiff):
    t1 = []
    t1_target=[]
    t2_target=[]
    columns =[2]
    columnnext=range(4,4+512)
    columnrest=range(5+512,5+512*2)
    columns.extend(columnnext)
    columns.extend(columnrest)
    #columns.extend(range(4,4+512)).extend(range(5+512,5+512*2))
    t1 = np.array(featureListSame.iloc[:,columns].values)
    t2 = np.array(featureListDiff.iloc[0:featureListSame.shape[0],columns].values)
    t1=np.concatenate((t2,t1))
    np.random.shuffle(t1)
    print(t1)
    return t1
t1 = myProcessData(featureListSame,featureListDiff)
print(t1.shape)

[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(143062, 1025)


# Getting data for the dataset

In [5]:
#Getting the matrix of total data
def GenerateRawData(t1):    
    dataMatrix = [] 
    
    dataMatrix = np.transpose(t1)     
    print ("Data Matrix Shape.."+ str(dataMatrix.shape))
    return dataMatrix

def GenerateTrainingTarget(rawTraining,TrainingPercent = 80):
    TrainingLen = int(math.ceil(len(rawTraining)*(TrainingPercent*0.01)))
    t           = rawTraining[:TrainingLen]
    #print(str(TrainingPercent) + "% Training Target Generated..")
    return t

def GenerateTrainingDataMatrix(rawData, TrainingPercent = 80):
    T_len = int(math.ceil(rawData.shape[0]*0.01*TrainingPercent))
    print("RawData"+str(rawData.shape))
    d2 = rawData[0:T_len,:]
    #print(str(TrainingPercent) + "% Training Data Generated..")
    return d2


#Generating the list of validation target values which is 10% of total length
#List of size 10% of the total length will be obtained.
def GenerateValTargetVector(rawData, ValPercent, TrainingCount): 
    valSize = int(math.ceil(rawData.shape[0]*ValPercent*0.01))
    V_End = TrainingCount + valSize
    t =rawData[TrainingCount:V_End-1]
    #print (str(ValPercent) + "% Val Target Data Generated..")
    return t

GenerateRawData(t1)

def GenerateValData(rawData, ValPercent, TrainingCount): 
    #ceil() returns ceiling value of x i.e the smallest integer not less than x.
    valSize = int(math.ceil(rawData.shape[0]*ValPercent*0.01))
    V_End = TrainingCount + valSize
    dataMatrix = rawData[TrainingCount:V_End-1,:]
    print("Validation Data Matrix"+str(dataMatrix.shape))  
    return dataMatrix

Data Matrix Shape..(1025, 143062)


# Fetch and Prepare Dataset

In [6]:
RawData = np.array(t1[:,1:])
RawTarget = np.array(t1[:,0:1])
print (RawData.shape)
print (RawTarget.shape)

(143062, 1024)
(143062, 1)


# Prepare Training Data

In [7]:
TrainingTarget = np.array(GenerateTrainingTarget(RawTarget,TrainingPercent))
TrainingData   = GenerateTrainingDataMatrix(RawData,TrainingPercent)
#TrainingData= np.transpose(TrainingData)
print(TrainingTarget.shape)
print(TrainingData.shape)

RawData(143062, 1024)
(114450, 1)
(114450, 1024)


# Prepare Validation Data

In [8]:
#We have used np.array for ValDataAct because it is of list type and we want
# it to be of array type
#print(len(TrainingTarget))
ValDataAct = np.array(GenerateValTargetVector(RawTarget,ValidationPercent, (len(TrainingTarget))))
ValData    = GenerateValData(RawData,ValidationPercent, (len(TrainingTarget)))
ValData = np.transpose(ValData)
print(ValDataAct.shape)
print(ValData.shape)

Validation Data Matrix(14306, 1024)
(14306, 1)
(1024, 14306)


# Prepare Test Data

In [9]:
TestDataAct = np.array(GenerateValTargetVector(RawTarget,TestPercent, (len(TrainingTarget)+len(ValDataAct))))
TestData = GenerateValData(RawData,TestPercent, (len(TrainingTarget)+len(ValDataAct)))
#TestData = np.transpose(TestData)
print(ValDataAct.shape)
print(ValData.shape)

Validation Data Matrix(14306, 1024)
(14306, 1)
(1024, 14306)


## Tensorflow Model Definition

In [10]:
# Defining Placeholder
inputTensor  = tf.placeholder(tf.float32, [None, 1024])
outputTensor = tf.placeholder(tf.float32, [None, 1])

In [11]:
NUM_HIDDEN_NEURONS_LAYER_1 = 1000
LEARNING_RATE = 0.03

# Initializing the weights to Normal Distribution
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape,stddev=0.01))

# Initializing the input to hidden layer weights
input_hidden_weights  = init_weights([1024, NUM_HIDDEN_NEURONS_LAYER_1])
# Initializing the hidden to output layer weights
hidden_output_weights = init_weights([NUM_HIDDEN_NEURONS_LAYER_1, 1])

# Computing values at the hidden layer
#print("inputTensor: "+str(inputTensor))
#print("input_hidden_weights: "+str(input_hidden_weights))
hidden_layer = tf.nn.relu(tf.matmul(inputTensor, input_hidden_weights))
# Computing values at the output layer
output_layer = tf.matmul(hidden_layer, hidden_output_weights)

# Defining Error Function
error_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output_layer, labels=outputTensor))

# Defining Learning Algorithm and Training Parameters
training = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(error_function)

# Prediction Function
prediction = output_layer

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



# Training the Model

In [ ]:
NUM_OF_EPOCHS = 2500
BATCH_SIZE = 200

training_accuracy = []

with tf.Session() as sess:
    
    # Set Global Variables ?
    tf.global_variables_initializer().run()

    for epoch in tqdm_notebook(range(NUM_OF_EPOCHS)):
        
        #Shuffle the Training Dataset at each epoch
        p = np.random.permutation(range(len(TrainingData)))
        TrainingData  = TrainingData[p]
        #processedTrainingLabel = processedTrainingLabel[p]
        
        # Start batch training
        for start in range(0, len(TrainingData), BATCH_SIZE):
            end = start + BATCH_SIZE
            sess.run(training, feed_dict={inputTensor: TrainingData[start:end], 
                                          outputTensor: TrainingTarget[start:end]})
        # Training accuracy for an epoch
        training_accuracy.append(np.mean(np.argmax(TrainingTarget, axis=1) ==
                             sess.run(prediction, feed_dict={inputTensor: TrainingData,
                                                             outputTensor: TrainingTarget})))
        
    # Testing
    predictedTestLabel = sess.run(prediction, feed_dict={inputTensor: TestData})
    print("predictedTestLabel :"+str(predictedTestLabel))

# Testing accuracy

In [ ]:
def accuracy_model(predicted_labels, actual_labels):
    count=0
    print("predicted_labels :"+ str(predicted_labels.shape))
    print("actual_labels :"+ str(actual_labels.shape))
    for i in range(len(predicted_labels)):
        if predicted_labels[i]==actual_labels[i]:
            count=count+1;
    accuracy= (count/len(predicted_labels))*100
    print("Accuracy: "+ str(accuracy))
    return accuracy
accuracy_model(predictedTestLabel,TestDataAct)